In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
fake_data =pd.read_csv("Fake.csv")
true_data = pd.read_csv("True.csv")


In [3]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
fake_data["class"] = 0
true_data['class'] = 1

In [5]:
fake_data.shape, true_data.shape 

((23481, 5), (21417, 5))

In [6]:
fake_data_test = fake_data.tail(10)
fake_data.drop(fake_data.tail(10).index, axis=0, inplace=True)
true_data_test = true_data.tail(10)
true_data.drop(true_data.tail(10).index, axis=0, inplace=True)


In [7]:
fake_data.shape, true_data.shape

((23471, 5), (21407, 5))

In [8]:
fake_data_test.shape, true_data_test.shape

((10, 5), (10, 5))

In [9]:
fake_data_test.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [10]:
data_merge = pd.concat([fake_data, true_data], axis=0)
data_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [11]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [12]:
data = data_merge.drop(['title','subject','date'], axis='columns')


In [13]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
data = data.sample(frac=1)

In [15]:
data.shape

(44878, 2)

In [16]:
data.head()

,text,class
19018,Liberal Delaware Sen. Chris Coons caused a sti...,0
10978,DC Antifa Leader Moved to Turkey with Man She ...,0
391,"On Tuesday afternoon, Miami-Dade Police tweete...",0
12561,"A stunning story of corruption, cronyism and c...",0
10141,Comey lied and has been caught in his lie What...,0


In [17]:
data.reset_index(inplace=True)

In [18]:
data.head()

,index,text,class
0,19018,Liberal Delaware Sen. Chris Coons caused a sti...,0
1,10978,DC Antifa Leader Moved to Turkey with Man She ...,0
2,391,"On Tuesday afternoon, Miami-Dade Police tweete...",0
3,12561,"A stunning story of corruption, cronyism and c...",0
4,10141,Comey lied and has been caught in his lie What...,0


In [19]:
data.drop(['index'], axis=1)#, inplace=True)

,text,class
0,Liberal Delaware Sen. Chris Coons caused a sti...,0
1,DC Antifa Leader Moved to Turkey with Man She ...,0
2,"On Tuesday afternoon, Miami-Dade Police tweete...",0
3,"A stunning story of corruption, cronyism and c...",0
4,Comey lied and has been caught in his lie What...,0
...,...,...
44873,WASHINGTON (Reuters) - The death of Cuba’s Fid...,1
44874,,0
44875,(Reuters) - Norfolk Southern Corp said on Tues...,1
44876,Trump on immigration appeal: We ll win. For t...,0


In [20]:
data.drop(['index'], axis=1, inplace=True)
data.head()

,text,class
0,Liberal Delaware Sen. Chris Coons caused a sti...,0
1,DC Antifa Leader Moved to Turkey with Man She ...,0
2,"On Tuesday afternoon, Miami-Dade Police tweete...",0
3,"A stunning story of corruption, cronyism and c...",0
4,Comey lied and has been caught in his lie What...,0


In [21]:
data.columns

Index(['text', 'class'], dtype='object')

In [22]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)    
    text = re.sub('[%s]' % re.escape(string.punctuation),'', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w*', '', text)
    return text

In [23]:
data['text']= data['text'].apply(wordopt)

In [24]:
data['text'][0]

'liberal delaware sen  chris coons caused a stir last week when he indicated during a televised interview that yet undisclosed transcripts of recorded phone conversations conclusively prove that elements of the trump campaign explicitly colluded with the russian government during the  presidential election  yet widely circulated  bombshell  reports from the new york times and cnn in recent weeks quoted sources who had no choice but to underline that there is no evidence of such coordination  and that the alleged contacts between trump allies and russian officials were not even necessarily unusual in nature  therefore  it s understandable why this statement from coons   a member of the senate foreign relations committee   touched off a fresh  feverish round of breathless speculation and rumors   townhall there are transcripts that provide very helpful  very critical insights into whether or not russian intelligence and senior russian political leaders  including vladimir putin  were coo

In [25]:
x = data['text']
y = data['class']

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
# instantiate the vectorizer object
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [28]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)


LogisticRegression()

In [29]:
pred_lr = LR.predict(xv_test)

In [30]:
LR.score(xv_test, y_test)

0.9844028520499108

In [31]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5910
           1       0.98      0.98      0.98      5310

    accuracy                           0.98     11220
   macro avg       0.98      0.98      0.98     11220
weighted avg       0.98      0.98      0.98     11220



In [32]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [33]:
pred_dt = DT.predict(xv_test)

In [34]:
DT.score(xv_test, y_test)

0.9939393939393939

In [35]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5910
           1       0.99      0.99      0.99      5310

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [36]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [37]:
import cupy as cp

In [38]:
pred_gb = GB.predict(xv_test)

In [39]:
GB.score(xv_test, y_test)

0.9946524064171123

In [40]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5910
           1       0.99      1.00      0.99      5310

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [41]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [42]:
pred_rf = GB.predict(xv_test)

In [43]:
RF.score(xv_test, y_test)

0.9884135472370766

In [44]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5910
           1       0.99      1.00      0.99      5310

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [49]:
def output_label(n):
    if n == 0:
        return "Fake News"
    else:
        return "Real News"
    
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {}  \nRFC Prediction: {}".format(output_label(pred_LR[0]),
                                                                                                               output_label(pred_DT[0]),
                                                                                                               output_label(pred_GB[0]),
                                                                                                               output_label(pred_RF[0])))

In [50]:
news = str(input())
manual_testing(news)



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News  
RFC Prediction: Real News


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
corpus = [
    'NLP is a way for computers to analyze, understand, and derive meaning from human language in a smart and useful way. By utilizing NLP, developers can organize and structure knowledge to perform tasks such as automatic summarization, translation, named entity recognition, relationship extraction, sentiment analysis, speech recognition, and topic segmentation.',
    'NLP is used to analyze text, allowing machines to understand how human’s speak. This human-computer interaction enables real-world applications like automatic text summarization, sentiment analysis, topic extraction, named entity recognition, parts-of-speech tagging, relationship extraction, stemming, and more. NLP is commonly used for text mining, machine translation, and automated question answering.',
    'NLP algorithms are typically based on machine learning algorithms. Instead of hand-coding large sets of rules, NLP can rely on machine learning to automatically learn these rules by analyzing a set of examples (i.e. a large corpus, like a book, down to a collection of sentences), and making a stastical inference. In general, the more data analyzed, the more accurate the model will be.',
]
vectorizer = CountVectorizer(stop_words='english', min_df=2, binary=False)
X = vectorizer.fit_transform(corpus)
docterm = pd.DataFrame(X.toarray(), index=['Doc1', 'Doc2', 'Doc3'], columns=list(vectorizer.get_feature_names()))

print(docterm.T)  

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [54]:
import sklearn
print(sklearn.__version__)

1.3.0
